# ================================
#
# TESTS SUR LES RUN.PY
#
# ================================

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [8]:
from preprocessing import full_preprocessing

X_train, Y_train, Y_train_oh, ids, X_test = full_preprocessing()



In [2]:
from run import generate_submission_from_model

ModuleNotFoundError: No module named 'tensorflow'

In [14]:
"""
import numpy as np
import pandas as pd
from our_functionsv3 import *

import tensorflow.keras as keras
from tensorflow import set_random_seed
set_random_seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.callbacks import EarlyStopping
from keras import regularizers
import time
import matplotlib
import matplotlib.pyplot as plt
"""

'\nimport numpy as np\nimport pandas as pd\nfrom our_functionsv3 import *\n\nimport tensorflow.keras as keras\nfrom tensorflow import set_random_seed\nset_random_seed(0)\nfrom keras.models import Model\nfrom keras.layers import Dense, Input, Dropout, LSTM, Activation\nfrom keras.layers.embeddings import Embedding\nfrom keras.preprocessing import sequence\nfrom keras.initializers import glorot_uniform\nfrom keras.callbacks import EarlyStopping\nfrom keras import regularizers\nimport time\nimport matplotlib\nimport matplotlib.pyplot as plt\n'

# ============================================
#
# OTHER TEST FOR THE RUN.py
#
# ============================================

In [3]:
from preprocessing import full_preprocessing
from run_fct import read_glove_vecs_only_alpha, get_max_length, sentences_to_indices

In [4]:
X_train, Y_train, Y_train_oh, ids, X_test = full_preprocessing()

word_to_index, index_to_word, word_to_vec_map = read_glove_vecs_only_alpha('dictionnary/glove.twitter.27B.200d.txt')

max_length = get_max_length(X_train)

X_train_indices = sentences_to_indices(X_train, word_to_index, max_length)
X_test_indices = sentences_to_indices(X_test, word_to_index, max_length)

52700 words were not in the dictionary
2698 words were not in the dictionary


In [5]:
def smiley_LSTM_improved(input_shape, word_to_vec_map, word_to_index, dropout_rate, hidden_layers, reg, l1_lambda, l2_lambda):
    
    if reg=='l2':
        regularizer = regularizers.l2(l2_lambda)
    elif reg=='l1':
        regularizer = regularizers.l1(l1_lambda)
    else:
        regularizer = None
        
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape = input_shape, dtype = 'int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)
    
    X = Conv2D(32, kernel_size=(3, 3), activation='relu') (embeddings)
    X = MaxPooling2D(pool_size=(2, 2)) (X)
    X = Conv2D(64, (3, 3), activation='relu') (X)
    X = Dropout(0.5)(X)
    X = Dense(128, activation='relu')
    X = Dropout(0.5)(X)    
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    '''
    X = LSTM(32, return_sequences = True, return_state = False, kernel_regularizer=regularizer)(X)
    X = Dropout(0.5)(X) 
    X = LSTM(32, return_sequences = True, return_state = False, kernel_regularizer=regularizer)(X)
    X = Dropout(0.5)(X)   
    X = LSTM(32, return_sequences = False, return_state = False)(X)
    X = Dropout(0.5)(X)
    '''
    
    # Propagate X through a Dense layer with softmax activation to get back a batch of 2-dimensional vectors.
    X = Dense(2)(X)
    
    # Add a softmax activation
    X = Activation('softmax')(X)

    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs = sentence_indices, outputs = X)
    
    return model

In [6]:
def complete_model_improved(X_train_indices, Y_train_oh, word_to_vec_map, word_to_index, max_length, summary = False, dropout_rate = 0.25, batch_size = 128, 
                   epochs = 50, loss ='categorical_crossentropy', optimizer ='adam', hidden_layers = 1, 
                            reg = '', l1_lambda = 0, l2_lambda = 0):
    
    model = smiley_LSTM_improved((max_length,), word_to_vec_map, word_to_index, dropout_rate, hidden_layers, reg, l1_lambda, l2_lambda)
    
    if summary:
        model.summary()
        
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    
    earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=3, verbose=1, mode='auto')
    callbacks_list = [earlystop]
    
    start = time.time()
    history_lstm = model.fit(X_train_indices, Y_train_oh, epochs = 50, callbacks=callbacks_list, batch_size = 32, validation_split = 0.1, shuffle=True)
    end = time.time()
    print("Model took {} seconds (which is {} minutes or {} hours) to train".format((end - start), (end - start)/60, (end - start)/3600))
    
    return history_lstm, model

In [7]:
import numpy as np
import pandas as pd
from plots_file import *
from our_functionsv3 import *

np.random.seed(0)
import tensorflow.keras as keras
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.callbacks import EarlyStopping
import time
import matplotlib
import matplotlib.pyplot as plt

l2_lambda = [0]
for l2 in l2_lambda:
    history, model = complete_model_improved(X_train_indices, Y_train_oh, word_to_vec_map, word_to_index, max_length, summary = True, dropout_rate = 50, batch_size = 128, epochs = 50, loss ='categorical_crossentropy', 
                                             optimizer = 'adam', hidden_layers = 0, reg = None, l1_lambda = 0, l2_lambda = 0)
    label = predict_lstm(model, X_test_indices)
    path = 'submissions/submission_model_cnn.csv'
    #create_csv_submission(ids, label, path)

ModuleNotFoundError: No module named 'tensorflow'

# =============================================
#
#  EN DESSOUS : des tests sur la fonction preprocessing 
#
# =============================================

In [2]:
# JE TESTE MA FONCTION full_preprocessing ICI
"""
print(X_train[0:4], "\n")
print(Y_train[0:4], "\n")
print(Y_train_oh[0:4], "\n")
print(ids[0:4], "\n")
print(X_test[0:4], "\n")

print(type(Y_train_oh))
print(type(Y_train))
print(type(X_train))
"""

'\nprint(X_train[0:4], "\n")\nprint(Y_train[0:4], "\n")\nprint(Y_train_oh[0:4], "\n")\nprint(ids[0:4], "\n")\nprint(X_test[0:4], "\n")\n\nprint(type(Y_train_oh))\nprint(type(Y_train))\nprint(type(X_train))\n'

In [47]:
#============================================================================
#============================================================================
####################### LA JE FAIS STEP BY STEP #############################
#============================================================================
#============================================================================

from preprocessing import *
from our_functionsv3 import convert_to_one_hot, get_test_data

path_pos = "twitter-datasets/train_pos.txt"
path_neg = "twitter-datasets/train_neg.txt"
path_test = "twitter-datasets/test_data.txt"

train_pos = read_data(path_pos)
train_neg = read_data(path_neg)

pos = pd.DataFrame(train_pos, columns=["tweet"])
neg = pd.DataFrame(train_neg, columns=["tweet"])

test = read_data(path_test)
test_pd = pd.DataFrame(test, columns=["tweet"])

train_pos_preprocessed = pre_process_tweets(pos)
train_neg_preprocessed = pre_process_tweets(neg)

################# SECOND PART ############

X_pos = list(train_pos_preprocessed.tweet)
X_neg = list(train_neg_preprocessed.tweet)
X_train = X_pos + X_neg

Y_pos = np.ones(len(X_pos), dtype = int)
Y_neg = np.zeros(len(X_neg), dtype = int)
Y_train = np.concatenate((Y_pos, Y_neg), axis = -1)

X_train, Y_train = shuffle(X_train, Y_train, random_state=52) #shuffle our training set
Y_train_oh = convert_to_one_hot(Y_train, C=2)

ids, _ = get_test_data(path_test)


test_preprocessed = pre_process_tweets(test_pd)
X_test = list(test_preprocessed.tweet)


In [49]:
######################## JE CHECK SI MA FONCTION A L'AIR OK !!!
X_train[0:4]
Y_train[0:4]
Y_train_oh[0:4]
X_test[0:8]

['sea doo pro sea scooter sports with the portable seascootersave air stay longer in the water and url',
 'user shucks well i work all week so now i ca not come cheer you on oh and put those batteries in your calculator',
 'i cant stay away from bug thats my baby',
 'user no lol im perfectly fine and not contagious anymore lmao',
 'whenever i fall asleep watching the tv i always wake up with a headache',
 'user he needs to get rid of that thing it scares me lol but he do not need a car either he needs drivers ed again',
 'its whatever in a terrible mood',
 'yesss rt user user thanks jordan i love you and i m gon na call you later']

In [16]:
type(train_pos_preprocessed)
train_pos_preprocessed.tweet[0:4]
type(train_pos_preprocessed.tweet[0])

str

In [26]:
#################################### LINK BETWEEN PART 1 AND 2 ##################################
X_pos_try = list(train_pos_preprocessed.tweet)
X_pos_try[0:8]

['user i dunno justin read my mention or not only justin and god knows about that but i hope you will follow me #believe',
 'because your logic is so dumb i wo not even crop out your name or your photo tsk url',
 'user just put casper in a box looved the battle #crakkbitch',
 'user user thanks sir do not trip lil mama just keep doin ya thang',
 'visiting my brother tmr is the bestest birthday gift eveerrr',
 'user yay #lifecompleted tweet facebook me to let me know please',
 'user #1dnextalbumtitle feel for you rollercoaster of life song cocept life #yolo becoming famous #followmeplz',
 'workin hard or hardly workin rt user at hardee s with my future coworker user']

In [10]:
################################### SECOND PART #####################################
from our_functionsv3 import read_data
from sklearn.utils import shuffle

pos = "twitter-datasets/train_pos_tokenize_not_hash.txt"
neg = "twitter-datasets/train_neg_tokenize_not_hash.txt"

test_path = "twitter-datasets/test_preprocessed_tokenize_not_hash.txt"

X_pos = read_data(pos)
X_neg = read_data(neg)
X_train = X_pos + X_neg

Y_pos = np.ones(len(X_pos), dtype = int)
Y_neg = np.zeros(len(X_neg), dtype = int)
Y_train = np.concatenate((Y_pos, Y_neg), axis = -1)

X_train_shuffled, Y_train_shuffled = shuffle(X_train, Y_train, random_state=52)

In [27]:
X_pos[0:4]

['user i dunno justin read my mention or not only justin and god knows about that but i hope you will follow me #believe',
 'because your logic is so dumb i wo not even crop out your name or your photo tsk url',
 'user just put casper in a box looved the battle #crakkbitch',
 'user user thanks sir do not trip lil mama just keep doin ya thang']